### Big Data - Group Project 
#### Cleaning data


In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkBasics').getOrCreate()
spark

In [2]:
#change configuration settings on Spark 
# change the value of the execute memory . Same as at work. 
conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '4g'),
                                        ('spark.app.name', 'Spark Updated Conf'), 
                                        ('spark.executor.cores', '4'), 
                                        ('spark.cores.max', '4'), 
                                        ('spark.driver.memory','4g')])

In [3]:
sc = spark.sparkContext
sc.defaultParallelism
sc._jsc.sc().getExecutorMemoryStatus().size()

1

In [4]:
#Check data exists in location
!hadoop fs -ls -d "<hdfs location>"

Java HotSpot(TM) 64-Bit Server VM warning: ignoring option MaxPermSize=512M; support was removed in 8.0
ls: `<hdfs location>': No such file or directory


In [12]:
# this is the right way to read a file - from HDFS in this case
%time df = spark.read.csv("/user/josefinabollini/GroupProject/games_CUBS_socks.csv", inferSchema=True, header=True)
import pyspark.sql.functions as f
df = df.withColumn('MONTH_ok',f.col('DAY'))
df = df.withColumn('DAY_ok',f.col('MONTH')).drop('MONTH','DAY')
df =  df.withColumnRenamed('MONTH_ok','MONTH').withColumnRenamed('DAY_ok','DAY')
df = df.withColumn('NEIGHBORHOOD',f.when(f.col('LOCATION') == 'Wrigley Field','Lakeview').otherwise('Armour Square'))
df = df.withColumn('nb_code',f.when(f.col('LOCATION') == 'Wrigley Field',6).otherwise(34))
df.show(5)

CPU times: user 3.45 ms, sys: 4.03 ms, total: 7.48 ms
Wall time: 7.25 s
+--------------------+----+-----------+-----+---+-------------+-------+
|            LOCATION|YEAR|       TEAM|MONTH|DAY| NEIGHBORHOOD|nb_code|
+--------------------+----+-----------+-----+---+-------------+-------+
|Guaranteed Rate F...|2019|WHITE SOCKS|    3| 20|Armour Square|     34|
|       Wrigley Field|2019|       CUBS|    3| 21|     Lakeview|      6|
|       Wrigley Field|2019|       CUBS|    3| 21|     Lakeview|      6|
|Guaranteed Rate F...|2019|WHITE SOCKS|    3| 21|Armour Square|     34|
|       Wrigley Field|2019|       CUBS|    3| 22|     Lakeview|      6|
+--------------------+----+-----------+-----+---+-------------+-------+
only showing top 5 rows



In [25]:
%time other_events_df = spark.read.csv("/user/josefinabollini/GroupProject/otherEvents.csv", inferSchema=True, header=True)
other_events_df =  other_events_df.withColumnRenamed('Location','LOCATION').withColumnRenamed('NEIGHBOURHOOD','NEIGHBORHOOD')
other_events_df.show(5)

CPU times: user 2.36 ms, sys: 5.35 ms, total: 7.71 ms
Wall time: 9.18 s
+-------------+---+-----+----+--------------+----------+-------+
|     LOCATION|DAY|MONTH|YEAR|  NEIGHBORHOOD|      TEAM|nb_code|
+-------------+---+-----+----+--------------+----------+-------+
|United Center| 11|   10|2019|NEAR WEST SIDE|BLACKHAWKS|     28|
|United Center| 12|   10|2019|NEAR WEST SIDE|BLACKHAWKS|     28|
|United Center| 15|   10|2019|NEAR WEST SIDE|BLACKHAWKS|     28|
|United Center| 19|   10|2019|NEAR WEST SIDE|BLACKHAWKS|     28|
|United Center| 20|   10|2019|NEAR WEST SIDE|BLACKHAWKS|     28|
+-------------+---+-----+----+--------------+----------+-------+
only showing top 5 rows



In [26]:
df = df.select("LOCATION","DAY","MONTH","YEAR","NEIGHBORHOOD",'TEAM','nb_code')
other_events_df = other_events_df.select("LOCATION","DAY","MONTH","YEAR","NEIGHBORHOOD",'TEAM','nb_code')

df3 = df.unionAll(other_events_df)
df3.describe().show()

+-------+--------------------+------------------+------------------+------------------+--------------+-----------+-----------------+
|summary|            LOCATION|               DAY|             MONTH|              YEAR|  NEIGHBORHOOD|       TEAM|          nb_code|
+-------+--------------------+------------------+------------------+------------------+--------------+-----------+-----------------+
|  count|                 582|               582|               582|               582|           582|        582|              582|
|   mean|                null|15.895189003436426| 6.733676975945017|2020.0618556701031|          null|       null|22.72680412371134|
| stddev|                null| 8.847591397903198|2.8855745232467673|0.9920277707270452|          null|       null|12.29804581650141|
|    min|Guaranteed Rate F...|                 1|                 1|              2019| Armour Square|      BEARS|                6|
|    max|       Wrigley Field|                31|                12| 

In [27]:
df3.toPandas().to_csv("/home/josefinabollini/GroupProject/All_events_aligned.csv")

### Weather Data - Cleaning


In [33]:
%time df21 = spark.read.csv("/user/josefinabollini/GroupProject/2021weatherdata.csv", inferSchema=True, header=True)
%time df22 = spark.read.csv("/user/josefinabollini/GroupProject/2022weatherdata.csv", inferSchema=True, header=True)
%time df1920 = spark.read.csv("/user/josefinabollini/GroupProject/2019-2020weatherdata.csv", inferSchema=True, header=True)



CPU times: user 2.76 ms, sys: 4.76 ms, total: 7.52 ms
Wall time: 8.17 s
CPU times: user 4.22 ms, sys: 1.06 ms, total: 5.27 ms
Wall time: 539 ms
CPU times: user 6.02 ms, sys: 0 ns, total: 6.02 ms
Wall time: 875 ms


In [31]:
df1920.show(10)

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-31-9825d92d2840>, line 1)